Import necessary libraries

In [29]:
import cv2  # For image processing
import numpy as np  # For matrix operations
import matplotlib.pyplot as plt  # For visualization
import urllib.request  # To download images from the internet
from urllib.error import HTTPError, URLError  # To handle errors

Function to download and load image from URL

In [30]:
def load_image_from_url(url, size=(300, 300)):
    """Downloads an image from a URL and resizes it. Returns None if URL fails."""
    try:
        resp = urllib.request.urlopen(url)
        image = np.asarray(bytearray(resp.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)

        # Check if image is valid before resizing
        if image is None:
            print(f"Failed to load image from {url}")
            return None

        # Resize and convert to RGB
        image = cv2.resize(image, size)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    except (HTTPError, URLError) as e:
        print(f"Error loading {url}: {e}")
        return None

URLs of the images

In [34]:
monument_url = "https://upload.wikimedia.org/wikipedia/commons/a/a8/Tour_Eiffel_Wikimedia_Commons.jpg"
mountain_url = "https://upload.wikimedia.org/wikipedia/commons/5/51/Matterhorn-600px.jpg"
animal_url = "https://upload.wikimedia.org/wikipedia/commons/7/73/Lion_waiting_in_Namibia.jpg"

# Load the images
monument = load_image_from_url(monument_url)
mountain = load_image_from_url(mountain_url)
animal = load_image_from_url(animal_url)

Validate images and use a black placeholder if any image fails

In [37]:
def validate_image(img, size=(300, 300)):
    """Returns a black placeholder image if the original fails."""
    if img is None:
        print("Using a black placeholder image.")
        return np.zeros((size[0], size[1], 3), dtype=np.uint8)
    return img

monument = validate_image(monument)
mountain = validate_image(mountain)
animal = validate_image(animal)

# Store images in a dictionary
images = {
    "Monument": monument,
    "Mountain": mountain,
    "Animal": animal
}

Function to apply convolution with a given kernel

In [38]:
# Convolution filter function for multi-channel images
def apply_filter_multichannel(image, kernel):
    """Applies a 2D filter to each channel of an image and combines the results."""
    filtered_channels = [cv2.filter2D(image[:, :, c], -1, kernel) for c in range(3)]
    return cv2.merge(filtered_channels)

# 2D Kernels
blur_kernel = np.ones((5, 5), np.float32) / 25  # Reduced the divisor for proper blur
sharpen_kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]], dtype=np.float32)
edge_kernel_x = np.array([[-1, 0, 1],
                          [-2, 0, 2],
                          [-1, 0, 1]], dtype=np.float32)
edge_kernel_y = np.array([[-1, -2, -1],
                          [0, 0, 0],
                          [1, 2, 1]], dtype=np.float32)

Apply filters to each image

In [39]:
results = {}
for name, img in images.items():
    results[name] = {
        "Original": img,
        "Blur": apply_filter_multichannel(img, blur_kernel),
        "Sharpen": apply_filter_multichannel(img, sharpen_kernel),
        "Edge": cv2.add(
            apply_filter_multichannel(img, edge_kernel_x),
            apply_filter_multichannel(img, edge_kernel_y)
        )
    }

Visualize results for each image

In [40]:
# Function to display results
def display_results(results):
    """Displays the original and filtered images side by side."""
    for name, filtered in results.items():
        fig, axes = plt.subplots(1, 4, figsize=(20, 6))

        axes[0].imshow(filtered["Original"])
        axes[0].set_title(f"{name} - Original")
        axes[0].axis('off')

        axes[1].imshow(filtered["Blur"])
        axes[1].set_title(f"{name} - Blurred")
        axes[1].axis('off')

        axes[2].imshow(filtered["Sharpen"])
        axes[2].set_title(f"{name} - Sharpened")
        axes[2].axis('off')

        axes[3].imshow(filtered["Edge"])
        axes[3].set_title(f"{name} - Edge Detection")
        axes[3].axis('off')

        plt.tight_layout()
        plt.show()

# Display results
display_results(results)

Output hidden; open in https://colab.research.google.com to view.